In [ ]:
import datetime as dt
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data path
data_path = '/content/drive/MyDrive/nba_final/Data'
os.chdir(data_path)

In [ ]:
#read csv file
df = pd.read_csv('players_stats_by_season_full_details.csv')

In [ ]:
# Only keep NBA from league column
df = df[df['League'] == 'NBA']
print(df['League'].value_counts())

League
NBA    7633
Name: count, dtype: int64


In [ ]:
df.head(345)

,League,Season,Stage,Player,Team,GP,MIN,FGM,FGA,3PM,...,birth_date,height,height_cm,weight,weight_kg,nationality,high_school,draft_round,draft_pick,draft_team
0,NBA,1999 - 2000,Regular_Season,Shaquille O'Neal,LAL,79,3163.0,956,1665,0,...,"Mar 6, 1972",7-1,216.0,325.0,147.0,United States,Robert G. Cole High School,1.0,1.0,Orlando Magic
1,NBA,1999 - 2000,Regular_Season,Vince Carter,TOR,82,3126.0,788,1696,95,...,"Jan 26, 1977",6-6,198.0,220.0,100.0,United States,Mainland High School,1.0,5.0,Golden State Warriors
2,NBA,1999 - 2000,Regular_Season,Karl Malone,UTA,82,2947.0,752,1476,2,...,"Jul 24, 1963",6-9,206.0,265.0,120.0,United States,Summerfield High School,1.0,13.0,Utah Jazz
3,NBA,1999 - 2000,Regular_Season,Allen Iverson,PHI,70,2853.0,729,1733,89,...,"Jun 7, 1975",6-0,183.0,165.0,75.0,United States,Bethel High School,1.0,1.0,Philadelphia Sixers
4,NBA,1999 - 2000,Regular_Season,Gary Payton,SEA,82,3425.0,747,1666,177,...,"Jul 23, 1968",6-4,193.0,180.0,82.0,United States,Skyline High School,1.0,2.0,Seattle SuperSonics
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,NBA,1999 - 2000,Playoffs,Randy Livingston,PHX,7,63.0,6,27,2,...,"Apr 2, 1975",6-4,193.0,209.0,95.0,United States,Isidore Newman School,2.0,13.0,Houston Rockets
341,NBA,1999 - 2000,Playoffs,Doug Christie,TOR,3,61.0,3,13,3,...,"May 9, 1970",6-6,198.0,200.0,91.0,United States,Rainier Beach High School,1.0,17.0,Seattle SuperSonics
505,NBA,2000 - 2001,Regular_Season,Jerry Stackhouse,DET,80,3215.0,774,1927,166,...,"Nov 5, 1974",6-6,198.0,218.0,99.0,United States,Oak Hill Academy,1.0,3.0,Philadelphia Sixers
506,NBA,2000 - 2001,Regular_Season,Allen Iverson,PHI,71,2979.0,762,1813,98,...,"Jun 7, 1975",6-0,183.0,165.0,75.0,United States,Bethel High School,1.0,1.0,Philadelphia Sixers


In [ ]:
print(df['League'].value_counts())

League
NBA    7633
Name: count, dtype: int64


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 7633 entries, 0 to 52115
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   League       7633 non-null   object 
 1   Season       7633 non-null   object 
 2   Stage        7633 non-null   object 
 3   Player       7633 non-null   object 
 4   Team         7633 non-null   object 
 5   GP           7633 non-null   int64  
 6   MIN          7633 non-null   float64
 7   FGM          7633 non-null   int64  
 8   FGA          7633 non-null   int64  
 9   3PM          7633 non-null   int64  
 10  3PA          7633 non-null   int64  
 11  FTM          7633 non-null   int64  
 12  FTA          7633 non-null   int64  
 13  TOV          7633 non-null   int64  
 14  PF           7633 non-null   int64  
 15  ORB          7633 non-null   int64  
 16  DRB          7633 non-null   int64  
 17  REB          7633 non-null   int64  
 18  AST          7633 non-null   int64  
 19  STL       

In [ ]:
df.tail()

,League,Season,Stage,Player,Team,GP,MIN,FGM,FGA,3PM,...,birth_date,height,height_cm,weight,weight_kg,nationality,high_school,draft_round,draft_pick,draft_team
52111,NBA,2019 - 2020,Playoffs,Wesley Iwundu,ORL,5,75.8,6,20,4,...,"Dec 20, 1994",6-7,201.0,195.0,88.0,United States,Spring Westfield High School,2.0,3.0,Orlando Magic
52112,NBA,2019 - 2020,Playoffs,Semi Ojeleye,BOS,13,122.2,7,28,5,...,"Dec 5, 1994",6-7,201.0,241.0,109.0,United States,Ottawa High School,2.0,7.0,Boston Celtics
52113,NBA,2019 - 2020,Playoffs,J.R. Smith,LAL,10,74.9,7,26,6,...,"Sep 9, 1985",6-6,198.0,225.0,102.0,United States,Saint Benedict's Preparatory School,1.0,18.0,New Orleans Pelicans
52114,NBA,2019 - 2020,Playoffs,Juwan Morgan,UTA,7,86.5,3,12,2,...,"Apr 17, 1997",6-8,203.0,232.0,105.0,United States,Waynesville High School,NaN,NaN,NaN
52115,NBA,2019 - 2020,Playoffs,Matisse Thybulle,PHI,4,75.3,3,7,1,...,"Mar 4, 1997",6-5,196.0,200.0,91.0,United States,Eastside Catholic High School,1.0,20.0,Boston Celtics


In [ ]:
#create a dataframe containing draft_round 1 and draft_pick 1,2,3,4,5,6,7,8,9,10
df_draft = df[(df['draft_round'] == 1) & (df['draft_pick'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]))]

In [ ]:
#drop the players that are born before 1965 because our data starts from 1999 season and those players are already out of their primes.
df_draft = df_draft[df_draft['birth_year'] >= 1965]

In [ ]:
df_draft.head(30)

,League,Season,Stage,Player,Team,GP,MIN,FGM,FGA,3PM,...,birth_date,height,height_cm,weight,weight_kg,nationality,high_school,draft_round,draft_pick,draft_team
0,NBA,1999 - 2000,Regular_Season,Shaquille O'Neal,LAL,79,3163.0,956,1665,0,...,"Mar 6, 1972",7-1,216.0,325.0,147.0,United States,Robert G. Cole High School,1.0,1.0,Orlando Magic
1,NBA,1999 - 2000,Regular_Season,Vince Carter,TOR,82,3126.0,788,1696,95,...,"Jan 26, 1977",6-6,198.0,220.0,100.0,United States,Mainland High School,1.0,5.0,Golden State Warriors
3,NBA,1999 - 2000,Regular_Season,Allen Iverson,PHI,70,2853.0,729,1733,89,...,"Jun 7, 1975",6-0,183.0,165.0,75.0,United States,Bethel High School,1.0,1.0,Philadelphia Sixers
4,NBA,1999 - 2000,Regular_Season,Gary Payton,SEA,82,3425.0,747,1666,177,...,"Jul 23, 1968",6-4,193.0,180.0,82.0,United States,Skyline High School,1.0,2.0,Seattle SuperSonics
5,NBA,1999 - 2000,Regular_Season,Jerry Stackhouse,DET,82,3148.0,619,1447,83,...,"Nov 5, 1974",6-6,198.0,218.0,99.0,United States,Oak Hill Academy,1.0,3.0,Philadelphia Sixers
6,NBA,1999 - 2000,Regular_Season,Grant Hill,DET,74,2776.0,696,1422,34,...,"Oct 5, 1972",6-8,203.0,225.0,102.0,United States,South Lakes High School,1.0,3.0,Detroit Pistons
7,NBA,1999 - 2000,Regular_Season,Kevin Garnett,MIN,81,3243.0,759,1526,30,...,"May 19, 1976",6-11,211.0,240.0,109.0,United States,Farragut Career Academy,1.0,5.0,Minnesota Timberwolves
9,NBA,1999 - 2000,Regular_Season,Chris Webber,SAC,75,2880.0,748,1548,27,...,"Mar 1, 1973",6-9,206.0,245.0,111.0,United States,Detroit Country Day School,1.0,1.0,Orlando Magic
10,NBA,1999 - 2000,Regular_Season,Ray Allen,MIL,82,3070.0,642,1411,172,...,"Jul 20, 1975",6-5,196.0,205.0,93.0,United States,Hillcrest High School,1.0,5.0,Minnesota Timberwolves
11,NBA,1999 - 2000,Regular_Season,Alonzo Mourning,MIA,79,2748.0,652,1184,0,...,"Feb 8, 1970",6-10,208.0,240.0,109.0,United States,Indian River High School,1.0,2.0,Charlotte Hornets


In [ ]:
df_draft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2626 entries, 0 to 52110
Data columns (total 34 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   League       2626 non-null   object 
 1   Season       2626 non-null   object 
 2   Stage        2626 non-null   object 
 3   Player       2626 non-null   object 
 4   Team         2626 non-null   object 
 5   GP           2626 non-null   int64  
 6   MIN          2626 non-null   float64
 7   FGM          2626 non-null   int64  
 8   FGA          2626 non-null   int64  
 9   3PM          2626 non-null   int64  
 10  3PA          2626 non-null   int64  
 11  FTM          2626 non-null   int64  
 12  FTA          2626 non-null   int64  
 13  TOV          2626 non-null   int64  
 14  PF           2626 non-null   int64  
 15  ORB          2626 non-null   int64  
 16  DRB          2626 non-null   int64  
 17  REB          2626 non-null   int64  
 18  AST          2626 non-null   int64  
 19  STL       

In [ ]:
#create a df with Player, draft_team to merge with the pir average later
df_draft_team = df[['Player', 'draft_team']]

In [ ]:
#get rid of unneeded columns
df_draft = df_draft.drop(['birth_year', 'birth_month', 'birth_date', 'height', 'height_cm', 'weight', 'weight_kg', 'nationality', 'high_school', 'Team', 'Season', 'Stage', 'draft_team', 'draft_round', 'draft_pick', 'League'], axis=1)

In [ ]:
df_draft.head()

,Player,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,PF,ORB,DRB,REB,AST,STL,BLK,PTS
0,Shaquille O'Neal,79,3163.0,956,1665,0,1,432,824,223,255,336,742,1078,299,36,239,2344
1,Vince Carter,82,3126.0,788,1696,95,236,436,551,178,263,150,326,476,322,110,92,2107
3,Allen Iverson,70,2853.0,729,1733,89,261,442,620,230,162,71,196,267,328,144,5,1989
4,Gary Payton,82,3425.0,747,1666,177,520,311,423,224,178,100,429,529,732,153,18,1982
5,Jerry Stackhouse,82,3148.0,619,1447,83,288,618,758,311,188,118,197,315,365,103,36,1939


In [ ]:
df_draft.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2626 entries, 0 to 52110
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  2626 non-null   object 
 1   GP      2626 non-null   int64  
 2   MIN     2626 non-null   float64
 3   FGM     2626 non-null   int64  
 4   FGA     2626 non-null   int64  
 5   3PM     2626 non-null   int64  
 6   3PA     2626 non-null   int64  
 7   FTM     2626 non-null   int64  
 8   FTA     2626 non-null   int64  
 9   TOV     2626 non-null   int64  
 10  PF      2626 non-null   int64  
 11  ORB     2626 non-null   int64  
 12  DRB     2626 non-null   int64  
 13  REB     2626 non-null   int64  
 14  AST     2626 non-null   int64  
 15  STL     2626 non-null   int64  
 16  BLK     2626 non-null   int64  
 17  PTS     2626 non-null   int64  
dtypes: float64(1), int64(16), object(1)
memory usage: 389.8+ KB


In [ ]:
#groupby the name so every other stat just sums up into a one total under one player names for their different season combinations
df_draft_sum = df_draft.groupby('Player').sum()

In [ ]:
df_draft_sum.head(30)

,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,PF,ORB,DRB,REB,AST,STL,BLK,PTS
Player,,,,,,,,,,,,,,,,,
Aaron Gordon,361,10883.9,1876,4184,436,1355,788,1131,537,749,604,1850,2454,949,297,236,4976
Adam Morrison,78,2325.9,355,944,87,258,120,169,130,179,54,176,230,163,28,6,917
Adonal Foyle,474,9223.5,927,1874,0,1,238,460,390,1121,879,1480,2359,227,188,835,2092
Al Horford,937,30792.5,5388,10458,534,1447,1587,2085,1465,2180,1948,5789,7737,3148,747,1099,12897
Al-Farouq Aminu,728,18428.6,2065,4899,605,1776,965,1296,893,1370,1043,3420,4463,904,693,422,5700
Alex Len,356,7290.6,1205,2432,95,285,690,999,477,957,767,1600,2367,359,153,338,3195
Allen Iverson,640,27247.9,6506,15332,790,2494,4974,6256,2292,1163,443,1917,2360,4006,1445,107,18776
Alonzo Mourning,345,9023.9,1670,3053,1,9,1147,1754,675,1109,720,1623,2343,264,105,926,4488
Amar'e Stoudemire,604,20438.9,4741,8838,29,125,3471,4574,1538,2084,1599,3640,5239,811,505,872,12982


In [ ]:
#add a missed fg column by calculating FGA - FGM
df_draft_sum['missed_fg'] = df_draft_sum['FGA'] - df_draft_sum['FGM']

In [ ]:
#add a missed_ft column by calculating FTA - FTM
df_draft_sum['missed_ft'] = df_draft_sum['FTA'] - df_draft_sum['FTM']

In [ ]:
#add a minutes played per game column by calculating MIN/GP
df_draft_sum['MPG'] = df_draft_sum['MIN']/df_draft_sum['GP']

In [ ]:
#add a PIR (player index rating) column by calculating: (PTS+AST+REB+STL+BLK-PF-MissFG-MissFT-TOV)/GP
df_draft_sum['PIR'] = (df_draft_sum['PTS']+df_draft_sum['AST']+df_draft_sum['REB']+df_draft_sum['STL']+df_draft_sum['BLK']-df_draft_sum['PF']-df_draft_sum['missed_fg']-df_draft_sum['missed_ft']-df_draft_sum['TOV'])/df_draft_sum['GP']

In [ ]:
df_draft_sum.head(50)

,GP,MIN,FGM,FGA,3PM,3PA,FTM,FTA,TOV,PF,...,DRB,REB,AST,STL,BLK,PTS,missed_fg,missed_ft,MPG,PIR
Player,,,,,,,,,,,,,,,,,,,,,
Aaron Gordon,361,10883.9,1876,4184,436,1355,788,1131,537,749,...,1850,2454,949,297,236,4976,2308,343,30.149307,13.781163
Adam Morrison,78,2325.9,355,944,87,258,120,169,130,179,...,176,230,163,28,6,917,589,49,29.819231,5.089744
Adonal Foyle,474,9223.5,927,1874,0,1,238,460,390,1121,...,1480,2359,227,188,835,2092,947,222,19.458861,6.373418
Al Horford,937,30792.5,5388,10458,534,1447,1587,2085,1465,2180,...,5789,7737,3148,747,1099,12897,5070,498,32.862860,17.518677
Al-Farouq Aminu,728,18428.6,2065,4899,605,1776,965,1296,893,1370,...,3420,4463,904,693,422,5700,2834,331,25.314011,9.277473
Alex Len,356,7290.6,1205,2432,95,285,690,999,477,957,...,1600,2367,359,153,338,3195,1227,309,20.479213,9.668539
Allen Iverson,640,27247.9,6506,15332,790,2494,4974,6256,2292,1163,...,1917,2360,4006,1445,107,18776,8826,1282,42.574844,20.517187
Alonzo Mourning,345,9023.9,1670,3053,1,9,1147,1754,675,1109,...,1623,2343,264,105,926,4488,1383,607,26.156232,12.614493
Amar'e Stoudemire,604,20438.9,4741,8838,29,125,3471,4574,1538,2084,...,3640,5239,811,505,872,12982,4097,1103,33.839238,19.183775


In [ ]:
df_pir_final = df_draft_sum.reset_index()[['Player', 'PIR', 'MPG']]

In [ ]:
df_pir_final.head(50)

,Player,PIR,MPG
0,Aaron Gordon,13.781163,30.149307
1,Adam Morrison,5.089744,29.819231
2,Adonal Foyle,6.373418,19.458861
3,Al Horford,17.518677,32.862860
4,Al-Farouq Aminu,9.277473,25.314011
5,Alex Len,9.668539,20.479213
6,Allen Iverson,20.517187,42.574844
7,Alonzo Mourning,12.614493,26.156232
8,Amar'e Stoudemire,19.183775,33.839238
9,Andre Drummond,20.471664,32.184644


In [ ]:
#join by using player name with df_draft_team so we can see pir, player name, mpg, and draft_team and also drop duplicate player name
df_pir_final = df_pir_final.join(df_draft_team.drop_duplicates(subset='Player').set_index('Player'), on='Player')

In [ ]:
df_pir_final.head(50)

,Player,PIR,MPG,draft_team
0,Aaron Gordon,13.781163,30.149307,Orlando Magic
1,Adam Morrison,5.089744,29.819231,Charlotte Hornets
2,Adonal Foyle,6.373418,19.458861,Golden State Warriors
3,Al Horford,17.518677,32.862860,Atlanta Hawks
4,Al-Farouq Aminu,9.277473,25.314011,Los Angeles Clippers
5,Alex Len,9.668539,20.479213,Phoenix Suns
6,Allen Iverson,20.517187,42.574844,Philadelphia Sixers
7,Alonzo Mourning,12.614493,26.156232,Charlotte Hornets
8,Amar'e Stoudemire,19.183775,33.839238,Phoenix Suns
9,Andre Drummond,20.471664,32.184644,Detroit Pistons


In [ ]:
# Scale the data (Z-scores)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_pir_final[['PIR', 'MPG']])

# Fit K-Means (Try k=3)
kmeans = KMeans(n_clusters=3, random_state=42)
df_pir_final['Cluster'] = kmeans.fit_predict(scaled_data)

In [ ]:
# scatterplot
fig = px.scatter(
    df_pir_final,
    x='PIR',
    y='MPG',
    color='Cluster',
    title="Player performance (MPG vs PIR)",
    hover_data=['Player','draft_team']
)

fig.update_layout(yaxis_range=[0, None], coloraxis_showscale=False)

fig

In [ ]:
fig.write_html("player_performance_kmeans.html")

In [ ]:
#show cluster 2
df_pir_final[df_pir_final['Cluster'] == 2]

,Player,PIR,MPG,draft_team,Cluster
2,Adonal Foyle,6.373418,19.458861,Golden State Warriors,2
5,Alex Len,9.668539,20.479213,Phoenix Suns,2
19,Antonio Daniels,8.176326,23.201726,Vancouver Grizzlies,2
21,Austin Rivers,5.752711,25.213232,New Orleans Pelicans,2
24,Ben McLemore,5.674267,25.349837,Sacramento Kings,2
...,...,...,...,...,...
268,Trey Burke,7.962547,24.471536,Minnesota Timberwolves,2
271,Tyrus Thomas,7.630081,19.738211,Portland Trail Blazers,2
277,Walt Williams,6.480349,21.091703,Sacramento Kings,2
279,Wesley Johnson,5.979127,22.886717,Minnesota Timberwolves,2


In [ ]:
#create a table having draft_team and the counts to be named underperformed pick number
underperformed_picks = df_pir_final[df_pir_final['Cluster'] == 2]['draft_team'].value_counts().reset_index()
underperformed_picks.columns = ['draft_team', 'underperformed pick number']
print(underperformed_picks)

                draft_team  underperformed pick number
0         Sacramento Kings                           8
1   Minnesota Timberwolves                           6
2        Charlotte Hornets                           5
3          New York Knicks                           5
4           Denver Nuggets                           5
5          Detroit Pistons                           4
6          Milwaukee Bucks                           4
7            Atlanta Hawks                           4
8     Los Angeles Clippers                           4
9       Washington Wizards                           4
10         Toronto Raptors                           3
11            Phoenix Suns                           3
12     Vancouver Grizzlies                           3
13           Orlando Magic                           3
14           Chicago Bulls                           2
15   Golden State Warriors                           2
16    New Orleans Pelicans                           2
17        

In [ ]:
#create a table graph of underperformed picks
fig2 = px.bar(underperformed_picks, x='draft_team', y='underperformed pick number', title='Underperforming Lottery Picks by Team (1999–2020)')

# Calculate the average of 'underperformed pick number'
average_underperformed = underperformed_picks['underperformed pick number'].mean()

# Add a horizontal line for the average
fig2.add_hline(y=average_underperformed, line_dash="dash", line_color="red", annotation_text=f"Average: {average_underperformed:.2f}", annotation_position="bottom right")

fig2.show()

In [ ]:
fig2.write_html("underperforming_lottery_picks.html")